In [1]:
NOTEBOOK_NAME = "e040_make_feat_kyouki_rate_by_one_col"

In [2]:
import os
import pandas as pd
import pickle
from tqdm.auto import tqdm
from sklearn.decomposition import TruncatedSVD

In [3]:
class Config:
    OUTPUT_DIR = f"../saved_data/{NOTEBOOK_NAME}"
    SEED = 33
    TARGET_COL = "reserve"


os.makedirs(Config.OUTPUT_DIR, exist_ok=True)

In [4]:
import os
import pandas as pd
import pickle
from tqdm.auto import tqdm
from sklearn.decomposition import TruncatedSVD

In [5]:
train_log = pd.read_csv("../data/train_log.csv")
train_label = pd.read_csv("../data/train_label.csv")

In [6]:
# e016にて、データ作成時にfoldを利用するように変更
with open(
    "../saved_data/e016_make_train_popular_base/session_id_fold_dict.pkl", "rb"
) as f:
    session_id_fold_dict = pickle.load(f)

In [7]:
train_label["fold"] = train_label["session_id"].map(session_id_fold_dict)

In [20]:
def make_kyouki_df(train_log: pd.DataFrame, train_label: pd.DataFrame):
    # latest_yad_noを追加する
    train_log = train_log.copy()
    train_label = train_label.copy()

    train_log_latest = (
        train_log.groupby("session_id")["yad_no"]
        .apply(lambda x: list(x)[-1])
        .rename("latest_yad_no")
    )

    train_label = train_label.merge(train_log_latest, on="session_id", how="left")
    train_latest_yad_no_yad_no_cnts = []
    for i in tqdm(range(5)):
        train_label_wo_fold = train_label[train_label["fold"] != i]

        train_latest_yad_no_yad_no_cnt = (
            train_label_wo_fold.groupby("latest_yad_no")["yad_no"]
            .value_counts()
            .reset_index()
        )
        train_latest_yad_no_yad_no_sum = train_label_wo_fold.groupby("latest_yad_no")[
            "yad_no"
        ].count()

        train_latest_yad_no_yad_no_cnt = train_latest_yad_no_yad_no_cnt.merge(
            train_latest_yad_no_yad_no_sum.rename("sum"), on="latest_yad_no", how="left"
        )

        train_latest_yad_no_yad_no_cnt["rate"] = (
            train_latest_yad_no_yad_no_cnt["count"]
            / train_latest_yad_no_yad_no_cnt["sum"]
        )
        train_latest_yad_no_yad_no_cnt.insert(0, "fold", i)

        train_latest_yad_no_yad_no_cnt = train_latest_yad_no_yad_no_cnt[
            ["fold", "latest_yad_no", "yad_no", "rate"]
        ]

        train_latest_yad_no_yad_no_cnts.append(train_latest_yad_no_yad_no_cnt)

    kyouki_cnt_df = pd.concat(train_latest_yad_no_yad_no_cnts)
    return kyouki_cnt_df


kyouki_train_df = make_kyouki_df(train_log, train_label)

  0%|          | 0/5 [00:00<?, ?it/s]

In [39]:
def make_kyouki_df_test(train_log: pd.DataFrame, train_label: pd.DataFrame):
    # latest_yad_noを追加する
    train_log = train_log.copy()
    train_label = train_label.copy()

    train_log_latest = (
        train_log.groupby("session_id")["yad_no"]
        .apply(lambda x: list(x)[-1])
        .rename("latest_yad_no")
    )

    train_label = train_label.merge(train_log_latest, on="session_id", how="left")

    train_latest_yad_no_yad_no_cnt = (
        train_label.groupby("latest_yad_no")["yad_no"].value_counts().reset_index()
    )

    train_label = train_label.merge(
        train_latest_yad_no_yad_no_cnt[["latest_yad_no", "count"]],
        on="latest_yad_no",
        how="left",
    )

    train_label_sum = (
        train_label.groupby("latest_yad_no")["yad_no"]
        .count()
        .rename("sum")
        .reset_index()
    )

    train_label = train_label.merge(
        train_label_sum[["latest_yad_no", "sum"]], on="latest_yad_no", how="left"
    )

    train_label["rate"] = train_label["count"] / train_label["sum"]

    train_label = train_label[["fold", "latest_yad_no", "yad_no", "rate"]]

    return train_label


kyouki_df_test = make_kyouki_df_test(train_log, train_label)

In [43]:
kyouki_train_df.to_pickle(f"{Config.OUTPUT_DIR}/kyouki_df_cnt_train.pkl")
kyouki_df_test.to_pickle(f"{Config.OUTPUT_DIR}/kyouki_df_cnt_test.pkl")

In [44]:
f"{Config.OUTPUT_DIR}/kyouki_df_cnt_train.pkl"

'../saved_data/e039_make_feat_kyouki_by_one_col/kyouki_df_cnt_train.pkl'